In [ ]:
!pip3 install selenium
!pip3 install bs4
!pip3 install pandas
!pip3 install xlwt
!pip3 install openpyxl

In [ ]:
import requests
import time
import bs4
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


In [ ]:
# Constant for setting script
key_word = "plant based"
show_only_shopee_mall = False

In [ ]:
# Helper Function
def zoom_out(driver): 
    driver.execute_script("document.getElementsByTagName('body')[0].style.scale='0.1';")
    driver.execute_script("document.getElementsByTagName('body')[0].style.transformOrigin = '0 0';")
    time.sleep(2)

def next_page(driver):
    next_button = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div/div/div[2]/div/div[1]/div[2]/button[2]')
    next_button.click()

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument("--incognito")
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)
driver.get("https://shopee.co.th/")

# Select Thai language
thai_button = driver.find_element(By.XPATH,'/html/body/div[2]/div[1]/div[1]/div/div[3]/div[1]/button')
thai_button.click()

# Close ads
ads_button = driver.execute_script('return document.querySelector(\"shopee-banner-popup-stateful\").shadowRoot.querySelector(\"div.shopee-popup__close-btn\")')
ads_button.click()

# Search product
search_input = driver.find_element(By.XPATH, '/html/body/div[1]/div/header/div[2]/div/div[1]/div[1]/div/form/input')
search_input.send_keys(key_word)
search_button = driver.find_element(By.XPATH, '/html/body/div[1]/div/header/div[2]/div/div[1]/div[1]/button')
search_button.click()

# Zoom out page
zoom_out(driver=driver)


# Check box Shopee Mall
if(show_only_shopee_mall):
    try:
        check_box = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div/div/div[1]/div[8]/div[2]/div[1]/div/label/span')
        check_box.click()
        time.sleep(3)
    except:
        print('Check box Shopee Mall not found')
        
# Find number of max pagination
max_page = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div/div/div[2]/div/div[1]/div[2]/div/span[2]')
max_page = int(max_page.text)


In [ ]:
# Get all product link
links = []

for n in range(max_page):
    print("get links page {} of {}".format(n + 1, max_page))

    if(n != 0):
        next_page(driver)
        zoom_out(driver)
    
    data = driver.page_source
    soup = bs4.BeautifulSoup(data)
    el=soup.select(".row>div")
    
    for e in el:
        for a in e.find_all('a', href=True):
            links.append(a['href'])
            
print("links count: {}".format(len(links)))

In [21]:
products = []
get_success = 0

# Open link and get detail products
for idx, link in enumerate(links):
    count = 1
    alway_get_detail = True
    
    print("product {} of {}: https://shopee.co.th{}".format(idx + 1, len(links), link))

    while alway_get_detail and count <= 3:
        try:
            driver.get("https://shopee.co.th" + link)
            time.sleep(3)

            data = driver.page_source
            soup = bs4.BeautifulSoup(data)

            name = soup.select_one("div.product-briefing > div:nth-child(3) > div:nth-child(1) > div:nth-child(1) > span").text.strip().replace(',', '')

            try:
                amount_text = soup.select_one('div.product-briefing > div:nth-child(3) > div:nth-child(1) > div:nth-child(2) > div:nth-child(3) > div:nth-child(1)').text.strip().replace("พัน","000").replace("ล้าน","000000")
                amount_float = float(re.sub('\D', '', amount_text))
            except:
                amount_float = 0
                
            company = soup.select_one('div.page-product__shop > div:nth-child(1) > div:nth-child(2) > div:nth-child(1)').text.strip()

            joined_text = soup.select_one('div.page-product__shop > div:nth-child(2) > div:nth-child(3) > div:nth-child(1) > span').text.strip()
            split_joined_text = re.split(' ', joined_text)
            joined = int(split_joined_text[0]) * int(split_joined_text[1].replace("ปี", "365").replace("เดือน", "30").replace("วัน", '1'))

            product_detail = soup.select_one('div.product-detail > div > div:nth-child(2)')
            product_detail = product_detail.select('div')
            for pd in product_detail:
                if(pd.select_one('label') and pd.select_one('label').text.strip() == 'ส่งจาก'):
                    detail = pd.select_one('div').text.strip().split(', ')
                    province = detail[len(detail) - 1]

            prices = []
            price = 0
            price_el = soup.select_one('div.product-briefing > div:nth-child(3) > div:nth-child(1) > div:nth-child(3) > div > div > div > div')
            price_el = price_el.select('div')
            for pd in price_el:
                if(re.search("^฿\d+(,\d{3})*(\.\d{1,2})?$" ,pd.text.strip())):
                    prices.append(float(pd.text.strip().replace('฿', '').replace(',', '')))
                
                price_text = pd.text.strip().split(' - ')
                if(len(price_text) == 2 and re.search("^฿\d+(,\d{3})*(\.\d{1,2})?$" ,price_text[0]) and re.search("^฿\d+(,\d{3})*(\.\d{1,2})?$" ,price_text[1])):
                    prices.append(float(price_text[0].replace('฿', '').replace(',', '')))
                    prices.append(float(price_text[1].replace('฿', '').replace(',', '')))

            if len(prices) != 0 :
                price = min(prices)
                
        except:
            print("can not get product {} of {}: {} times".format(idx + 1, len(links), count))
            count = count + 1
            continue

        get_success = get_success + 1
        alway_get_detail = False
        products.append([name, price, amount_float, company, province, joined])

print("successfully {} of {}".format(get_success, len(links)))

can not get product 305 of 728: 1 times


In [ ]:
import pandas as pd

# Exprot data to xlsx and json
df = pd.DataFrame(products, columns=['name', 'price', 'amount', 'company', 'province', 'joined'])
df.to_excel('plant_based_products.xlsx', encoding='utf-8')
df.to_json('plant_based_products.json', force_ascii=False)

In [ ]:
# For upload json to data node
from dsmlibrary.datanode import DataNode

dir_raw_id = 85
datanode = DataNode()
datanode.upload_file(directory_id=dir_raw_id, file_path='plant_based_products.json')